### Business Goal: To improve the efficiency of electronic taxi dispatching systems it is important to be able to predict how long a driver will have his taxi occupied.If a dispatcher knew approximately when a taxi driver would be ending their current ride, they would be better able to identify which driver to assign to each pickup request.

### ML Goal : To build model that predicts the total ride duration of taxi trips in New York City

### Data: The taxi trip records include fields capturing pick-up and drof-off dates/times, pick-up and drop-off locations, trip distances, itemized fares , rate types, payment types, and driver-reported passenger counts. The data used in the attached datasets were collected and provided to the NYC Taxi and Limousine Commission(TLC) by technology providers authorised under the Taxicab and Livery Passenger Enhancement Programs (TPEP/LPEP)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ASUS\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ASUS\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ASUS\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ASUS\Anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\

AttributeError: _ARRAY_API not found

In [2]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_parquet('yellow_tripdata_2024-01.parquet', engine='pyarrow')

In [4]:
print(df.head(500))

     VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0           2  2024-01-01 00:57:55   2024-01-01 01:17:43              1.0   
1           1  2024-01-01 00:03:00   2024-01-01 00:09:36              1.0   
2           1  2024-01-01 00:17:06   2024-01-01 00:35:01              1.0   
3           1  2024-01-01 00:36:38   2024-01-01 00:44:56              1.0   
4           1  2024-01-01 00:46:51   2024-01-01 00:52:57              1.0   
..        ...                  ...                   ...              ...   
495         2  2024-01-01 00:54:15   2024-01-01 01:31:18              1.0   
496         1  2024-01-01 00:39:14   2024-01-01 01:01:18              1.0   
497         2  2024-01-01 00:09:26   2024-01-01 00:19:31              1.0   
498         2  2024-01-01 00:23:16   2024-01-01 00:26:37              1.0   
499         2  2024-01-01 00:29:11   2024-01-01 00:32:33              1.0   

     trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocation

In [5]:
print(df.isnull().sum())

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          140162
trip_distance                 0
RatecodeID               140162
store_and_fwd_flag       140162
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     140162
Airport_fee              140162
dtype: int64


In [6]:
df = df.drop('RatecodeID', axis=1)

In [8]:
df['Airport_fee'].fillna(0, inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_31292\3756290478.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Airport_fee'].fillna(0, inplace=True)


In [9]:
print(df['Airport_fee'].isnull().sum())

0


In [10]:
print(df.isnull().sum())

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          140162
trip_distance                 0
store_and_fwd_flag       140162
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     140162
Airport_fee                   0
dtype: int64


In [16]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10


In [19]:
# Convert pickup and dropoff times to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Create 'trip_duration' in minutes
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Check the new column
print(df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_duration']].head())


  tpep_pickup_datetime tpep_dropoff_datetime  trip_duration
0  2024-01-01 00:57:55   2024-01-01 01:17:43      19.800000
1  2024-01-01 00:03:00   2024-01-01 00:09:36       6.600000
2  2024-01-01 00:17:06   2024-01-01 00:35:01      17.916667
3  2024-01-01 00:36:38   2024-01-01 00:44:56       8.300000
4  2024-01-01 00:46:51   2024-01-01 00:52:57       6.100000


In [18]:
print(df.isnull().sum())

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          140162
trip_distance                 0
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
dtype: int64


In [20]:

X = df[['VendorID', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID',
        'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]

y = df['trip_duration']

print(X.isnull().sum())


VendorID                      0
passenger_count          140162
trip_distance                 0
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
dtype: int64


In [21]:
# Fill missing passenger_count values with the median
X['passenger_count'].fillna(X['passenger_count'].median(), inplace=True)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_31292\4143002175.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['passenger_count'].fillna(X['passenger_count'].median(), inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_31292\4143002175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['passenger_count'].fillna(X['passenger_count']

In [22]:
from sklearn.model_selection import train_test_split

# Split the dataset: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit on training data and transform both training and test sets
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [24]:
from sklearn.linear_model import LinearRegression

# Initialize the linear regression model
model = LinearRegression()

# Train the model on the training data
model.fit(X_train, y_train)


LinearRegression()

In [25]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate R² score
r2 = r2_score(y_test, y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")


R² Score: 0.0759312976139006
Mean Squared Error: 984.0370885447893


In [26]:
# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming y_test and y_pred are already defined from your trained model
plt.figure(figsize=(10, 6))

# Scatter plot for actual vs predicted trip duration
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5, color='blue')
plt.title('Actual vs Predicted Trip Duration')
plt.xlabel('Actual Trip Duration (minutes)')
plt.ylabel('Predicted Trip Duration (minutes)')
plt.grid(True)

# Display the plot
plt.show()


AttributeError: module 'matplotlib.cm' has no attribute 'register_cmap'

In [30]:
!pip install --upgrade matplotlib


In [31]:
!pip install --upgrade seaborn


  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2


In [33]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming y_test and y_pred are defined
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5, color='blue')
plt.title('Actual vs Predicted Trip Duration')
plt.xlabel('Actual Trip Duration (minutes)')
plt.ylabel('Predicted Trip Duration (minutes)')
plt.grid(True)

plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_31292\969834644.py", line 2, in <cell line: 2>
    import seaborn as sns
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\__init__.py", line 9, in <module>
    from .matrix import *  # noqa: F401,F403
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\matrix.py", line 16, in <module>
    from . import cm
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\cm.py", line 2, in <module>
    from seaborn._compat import register_colormap
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\_compat.py", line 8, in <module>
    from seaborn.utils import _version_predates
ImportError: cannot import name '_version_predates' from 'seaborn.utils' (C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\utils.py)

During handling of t

In [34]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, r2_score

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5, color='blue')


plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')


plt.title('Actual vs Predicted Trip Duration')
plt.xlabel('Actual Trip Duration (minutes)')
plt.ylabel('Predicted Trip Duration (minutes)')
plt.grid(True)


mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
plt.text(0.05, 0.95, f'MAE: {mae:.2f}\nR²: {r2:.2f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')

plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ASUS\AppData\Local\Temp\ipykernel_31292\369586840.py", line 2, in <cell line: 2>
    import seaborn as sns
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\__init__.py", line 9, in <module>
    from .matrix import *  # noqa: F401,F403
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\matrix.py", line 16, in <module>
    from . import cm
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\cm.py", line 2, in <module>
    from seaborn._compat import register_colormap
  File "C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\_compat.py", line 8, in <module>
    from seaborn.utils import _version_predates
ImportError: cannot import name '_version_predates' from 'seaborn.utils' (C:\Users\ASUS\Anaconda3\lib\site-packages\seaborn\utils.py)

During handling of t